## Extracting Heights from pediatric FHIR patients

In [1]:
import requests
import base64
import json

# Configuration
BASE_URL = "http://localhost:8080/csp/healthshare/demo/fhir/r4"
USERNAME = "_System"
PASSWORD = "ISCDEMO"
HEADERS = {
    "Authorization": "Basic " + base64.b64encode(f"{USERNAME}:{PASSWORD}".encode()).decode(),
    "Accept": "*/*",
    "Content-Type": "application/fhir+json",
    "Accept-Encoding": "gzip, deflate, br",
    "Prefer": "return=representation"
}


### Install FHIRpath library

In [4]:
%pip install fhirpathpy
%pip install fhir.resources



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from fhir.resources.medicationstatement import MedicationStatement
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.fhirtypes import DateTime
from fhir.resources.dosage import Dosage

In [6]:

def build_medication_statement(
    subject: str,
    status: str,
    date_asserted: str,
    medication_system: str,
    medication_code: str,
    medication_display: str,
    dosage_text: str = None,
):
    """
    Build a FHIR MedicationStatement resource using fhir.resources.

    Parameters
    ----------
    subject : str
        Reference to the patient, e.g., "Patient/123".
    status : str
        FHIR status (e.g., "active", "completed", "entered-in-error").
    date_asserted : str
        ISO8601 datetime string, e.g. "2025-03-14T12:00:00Z".
    medication_system : str
        Coding system for the medication, e.g., "http://www.nlm.nih.gov/research/umls/rxnorm".
    medication_code : str
        Code for the medication, e.g., "617314".
    medication_display : str
        Human-friendly label, e.g., "Metformin 500 MG Oral Tablet".
    dosage_text : str
        Free text dose instruction, e.g., "Take 1 tablet (500 mg) by mouth twice daily."

    Returns
    -------
    dict
        A JSON-serializable dictionary representing the MedicationStatement.
    """

    med_codeable = CodeableConcept(
        coding=[
            Coding(
                system=medication_system,
                code=medication_code,
                display=medication_display
            )
        ]
    )

    dosage = None
    if dosage_text:
        dosage = [Dosage(text=dosage_text)]

    ms = MedicationStatement(
        status=status,
        subject={"reference": subject},
        dateAsserted=DateTime.validate(date_asserted),
        medicationCodeableConcept=med_codeable,
        dosage=dosage,
    )

    return ms.dict()  # or ms.json() if you want a JSON string


In [ ]:
ms_json = build_medication_statement(
    subject="Patient/7",
    status="active",
    date_asserted="2025-02-10T10:30:00Z",
    medication_system="http://www.nlm.nih.gov/research/umls/rxnorm",
    medication_code="617311",
    medication_display="atorvastatin 40 MG Oral Tablet",
    dosage_text="Take one 40 mg tablet by mouth once daily, with or without food."
)

ms_json


{'dateAsserted': datetime.datetime(2025, 2, 10, 10, 30, tzinfo=datetime.timezone.utc),
 'dosage': [{'text': 'Take one tablet twice daily with meals.'}],
 'medicationCodeableConcept': {'coding': [{'code': '617314',
    'display': 'Metformin 500 MG Oral Tablet',
    'system': 'http://www.nlm.nih.gov/research/umls/rxnorm'}]},
 'status': 'active',
 'subject': {'reference': 'Patient/7'},
 'resourceType': 'MedicationStatement'}

### Post the MedicalStatement to the FHIR Server

In [10]:
# Build resource object
ms = MedicationStatement(**ms_json)
BASE_URL = "http://localhost:8080/csp/healthshare/demo/fhir/r4/MedicationStatement"
# Send JSON directly
response = requests.post(
    BASE_URL,
    headers=HEADERS,
    data=ms.json()  # <-- this converts datetime → ISO string
)

print(response.status_code)
print(response.text)


201
{"dateAsserted":"2025-02-10T10:30:00+00:00","dosage":[{"text":"Take one tablet twice daily with meals."}],"medicationCodeableConcept":{"coding":[{"code":"617314","display":"Metformin 500 MG Oral Tablet","system":"http://www.nlm.nih.gov/research/umls/rxnorm"}]},"status":"active","subject":{"reference":"Patient/7"},"resourceType":"MedicationStatement","id":"70953","meta":{"lastUpdated":"2025-12-18T12:13:05Z","versionId":"1"}}


#### Lets try and create another medication statement for ASA 80mg QD, look up the rxnorm code: https://mor.nlm.nih.gov/RxNav

In [11]:
ms_json = build_medication_statement(
    subject="Patient/7",
    status="active",
    date_asserted="2025-04-10T10:30:00Z",
    medication_system="http://www.nlm.nih.gov/research/umls/rxnorm",
    medication_code="307696",
    medication_display="acetaminophen 80 MG Chewable Tablet",
    dosage_text="Take one tablet by mouth once daily, with or without food."
)

ms_json




{'dateAsserted': datetime.datetime(2025, 4, 10, 10, 30, tzinfo=datetime.timezone.utc),
 'dosage': [{'text': 'Take one tablet by mouth once daily, with or without food.'}],
 'medicationCodeableConcept': {'coding': [{'code': '307696',
    'display': 'acetaminophen 80 MG Chewable Tablet',
    'system': 'http://www.nlm.nih.gov/research/umls/rxnorm'}]},
 'status': 'active',
 'subject': {'reference': 'Patient/7'},
 'resourceType': 'MedicationStatement'}

#### Post to the FHIR Server

In [12]:
# Build resource object
ms = MedicationStatement(**ms_json)
BASE_URL = "http://localhost:8080/csp/healthshare/demo/fhir/r4/MedicationStatement"
# Send JSON directly
response = requests.post(
    BASE_URL,
    headers=HEADERS,
    data=ms.json()  # <-- this converts datetime → ISO string
)

print(response.status_code)
print(response.text)


201
{"dateAsserted":"2025-04-10T10:30:00+00:00","dosage":[{"text":"Take one tablet by mouth once daily, with or without food."}],"medicationCodeableConcept":{"coding":[{"code":"307696","display":"acetaminophen 80 MG Chewable Tablet","system":"http://www.nlm.nih.gov/research/umls/rxnorm"}]},"status":"active","subject":{"reference":"Patient/7"},"resourceType":"MedicationStatement","id":"70954","meta":{"lastUpdated":"2025-12-18T12:59:10Z","versionId":"1"}}


#### Retrieve all MedicationStatements for a given patient

In [13]:
BASE_URL = "http://localhost:8080/csp/healthshare/demo/fhir/r4/"

def get_medication_statements(patient_id: str):
    """
    Retrieve all the medication statements for a patient
    """

    params = {
        "subject": f"Patient/{patient_id}",
    }

    response = requests.get(
        f"{BASE_URL}/MedicationStatement",
        headers=HEADERS,
        params=params
    )

    return response.json()





#### Test this out on Patient/7

In [15]:
ms= get_medication_statements("7")
print(ms)

{'resourceType': 'Bundle', 'id': '3ee6378b-dc12-11f0-8a9c-ea5b7ebf2bb6', 'type': 'searchset', 'timestamp': '2025-12-18T13:05:37Z', 'total': 2, 'link': [{'relation': 'self', 'url': 'http://localhost:8080/csp/healthshare/demo/fhir/r4/MedicationStatement?subject=Patient%2F7'}], 'entry': [{'fullUrl': 'http://localhost:8080/csp/healthshare/demo/fhir/r4/MedicationStatement/70953', 'resource': {'dateAsserted': '2025-02-10T10:30:00+00:00', 'dosage': [{'text': 'Take one tablet twice daily with meals.'}], 'medicationCodeableConcept': {'coding': [{'code': '617314', 'display': 'Metformin 500 MG Oral Tablet', 'system': 'http://www.nlm.nih.gov/research/umls/rxnorm'}]}, 'status': 'active', 'subject': {'reference': 'Patient/7'}, 'resourceType': 'MedicationStatement', 'id': '70953', 'meta': {'lastUpdated': '2025-12-18T12:13:05Z', 'versionId': '1'}}, 'search': {'mode': 'match'}}, {'fullUrl': 'http://localhost:8080/csp/healthshare/demo/fhir/r4/MedicationStatement/70954', 'resource': {'dateAsserted': '202

#### Show this as a table

In [16]:
import pandas as pd

# bundle_json should be the dict you already printed
bundle = ms  # rename for clarity

rows = []

for entry in bundle.get("entry", []):
    resource = entry.get("resource", {})

    medication = resource.get("medicationCodeableConcept", {}).get("coding", [{}])[0]

    row = {
        "id": resource.get("id"),
        "status": resource.get("status"),
        "dateAsserted": resource.get("dateAsserted"),
        "medication_code": medication.get("code"),
        "medication_display": medication.get("display"),
        "medication_system": medication.get("system"),
        "dosage_text": resource.get("dosage", [{}])[0].get("text"),
        "patient_reference": resource.get("subject", {}).get("reference"),
    }

    rows.append(row)

df = pd.DataFrame(rows)
df


,id,status,dateAsserted,medication_code,medication_display,medication_system,dosage_text,patient_reference
0,70953,active,2025-02-10T10:30:00+00:00,617314,Metformin 500 MG Oral Tablet,http://www.nlm.nih.gov/research/umls/rxnorm,Take one tablet twice daily with meals.,Patient/7
1,70954,active,2025-04-10T10:30:00+00:00,307696,acetaminophen 80 MG Chewable Tablet,http://www.nlm.nih.gov/research/umls/rxnorm,"Take one tablet by mouth once daily, with or w...",Patient/7
